In [1]:
# Install required packages
!pip install gdown tqdm matplotlib pillow einops

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from einops import rearrange, repeat
import math
import random
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
import zipfile
import shutil
import glob
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# -------------------------
# Dataset Download and Setup - No changes needed
# -------------------------
def download_and_setup_dataset(force_download=False):
    """Download and properly set up Kvasir-SEG dataset"""
    base_path = './datasets'
    kvasir_path = os.path.join(base_path, 'kvasir-seg')

    # Make sure base directory exists
    os.makedirs(base_path, exist_ok=True)

    # Check if dataset already exists in the expected directory structure
    if os.path.exists(os.path.join(kvasir_path, 'images')) and \
       os.path.exists(os.path.join(kvasir_path, 'masks')) and \
       len(os.listdir(os.path.join(kvasir_path, 'images'))) > 0 and \
       not force_download:
        print("Kvasir-SEG dataset already exists.")
        return kvasir_path

    # Direct URL to the zip file
    dataset_url = "https://datasets.simula.no/downloads/kvasir-seg.zip"
    zip_path = os.path.join(base_path, 'kvasir-seg.zip')

    # Download the dataset
    print("Downloading Kvasir-SEG dataset...")
    try:
        response = requests.get(dataset_url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

        with open(zip_path, 'wb') as f:
            for data in response.iter_content(block_size):
                progress_bar.update(len(data))
                f.write(data)
        progress_bar.close()
    except Exception as e:
        print(f"Error downloading dataset: {e}")
        return None

    print(f"Download completed, file saved to {zip_path}")

    # Extract the dataset
    print("Extracting dataset...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(base_path)
        print("Extraction completed.")
    except Exception as e:
        print(f"Error extracting dataset: {e}")
        return None

    # Check the structure of extracted files
    extracted_files = glob.glob(os.path.join(base_path, "**"), recursive=True)
    print("Extracted file structure:")
    for file in extracted_files[:10]:  # Show only first 10 files
        print(f"  {file}")
    if len(extracted_files) > 10:
        print(f"  ... and {len(extracted_files)-10} more files")

    # Locate the images and masks directories
    image_dirs = glob.glob(os.path.join(base_path, "**/images"), recursive=True)
    mask_dirs = glob.glob(os.path.join(base_path, "**/masks"), recursive=True)

    print(f"Found image directories: {image_dirs}")
    print(f"Found mask directories: {mask_dirs}")

    # Ensure proper directory structure
    os.makedirs(os.path.join(kvasir_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(kvasir_path, 'masks'), exist_ok=True)

    # Copy files to the expected location if needed
    if image_dirs and mask_dirs:
        src_image_dir = image_dirs[0]
        src_mask_dir = mask_dirs[0]

        if src_image_dir != os.path.join(kvasir_path, 'images'):
            print(f"Moving images from {src_image_dir} to {os.path.join(kvasir_path, 'images')}")
            for img_file in os.listdir(src_image_dir):
                shutil.copy(
                    os.path.join(src_image_dir, img_file),
                    os.path.join(kvasir_path, 'images', img_file)
                )

        if src_mask_dir != os.path.join(kvasir_path, 'masks'):
            print(f"Moving masks from {src_mask_dir} to {os.path.join(kvasir_path, 'masks')}")
            for mask_file in os.listdir(src_mask_dir):
                shutil.copy(
                    os.path.join(src_mask_dir, mask_file),
                    os.path.join(kvasir_path, 'masks', mask_file)
                )

    # Clean up
    try:
        os.remove(zip_path)
        print("Removed zip file.")
    except:
        print("Could not remove zip file.")

    # Verify the dataset is now properly set up
    if os.path.exists(os.path.join(kvasir_path, 'images')) and \
       os.path.exists(os.path.join(kvasir_path, 'masks')) and \
       len(os.listdir(os.path.join(kvasir_path, 'images'))) > 0:
        print("Dataset setup completed successfully.")
        print(f"Found {len(os.listdir(os.path.join(kvasir_path, 'images')))} images and "
              f"{len(os.listdir(os.path.join(kvasir_path, 'masks')))} masks.")
        return kvasir_path
    else:
        print("Dataset setup failed.")
        return None

# -------------------------
# Dataset class with augmentation
# -------------------------
class KvasirSEGDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None, target_transform=None, augment=True):
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.augment = augment and split == 'train'  # Only augment training data
        self.img_dir = os.path.join(root_dir, 'images')
        self.mask_dir = os.path.join(root_dir, 'masks')

        # Verify directories exist
        if not os.path.exists(self.img_dir):
            raise ValueError(f"Images directory not found: {self.img_dir}")
        if not os.path.exists(self.mask_dir):
            raise ValueError(f"Masks directory not found: {self.mask_dir}")

        # Get all image files
        self.images = sorted([f for f in os.listdir(self.img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
        if not self.images:
            raise ValueError(f"No images found in {self.img_dir}")

        # Print some sample image names for debugging
        print(f"Sample image names: {self.images[:5]}")

        # Split data into train/val/test (80/10/10 split)
        np.random.seed(42)  # For reproducibility
        indices = np.random.permutation(len(self.images))

        if split == 'train':
            self.images = [self.images[i] for i in indices[:int(0.8 * len(self.images))]]
        elif split == 'val':
            self.images = [self.images[i] for i in indices[int(0.8 * len(self.images)):int(0.9 * len(self.images))]]
        else:  # test
            self.images = [self.images[i] for i in indices[int(0.9 * len(self.images)):]]

        print(f"Created {split} dataset with {len(self.images)} images")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)

        # Find corresponding mask
        base_name = os.path.splitext(img_name)[0]
        mask_candidates = [
            os.path.join(self.mask_dir, base_name + ext)
            for ext in ['.jpg', '.png', '.jpeg', '.tif']
        ]
        mask_path = next((path for path in mask_candidates if os.path.exists(path)), None)

        if not mask_path:
            # Look for files that start with the same name
            mask_files = os.listdir(self.mask_dir)
            matches = [f for f in mask_files if f.startswith(base_name)]
            if matches:
                mask_path = os.path.join(self.mask_dir, matches[0])
            else:
                raise FileNotFoundError(f"No mask found for image {img_name}")

        # Print paths for debugging (only for the first item)
        if idx == 0:
            print(f"Image path: {img_path}")
            print(f"Mask path: {mask_path}")

        # Load image and mask
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Apply augmentation if enabled
        if self.augment:
            # Random horizontal flip
            if random.random() > 0.5:
                image = TF.hflip(image)
                mask = TF.hflip(mask)

            # Random vertical flip
            if random.random() > 0.5:
                image = TF.vflip(image)
                mask = TF.vflip(mask)

            # Random rotation
            if random.random() > 0.5:
                angle = random.choice([90, 180, 270])
                fill = 0
                image = TF.rotate(image, angle, fill=fill)
                mask = TF.rotate(mask, angle, fill=fill)

            # Color jitter (only for image)
            if random.random() > 0.5:
                image = TF.adjust_brightness(image, random.uniform(0.8, 1.2))
                image = TF.adjust_contrast(image, random.uniform(0.8, 1.2))
                image = TF.adjust_saturation(image, random.uniform(0.8, 1.2))

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            mask = self.target_transform(mask)
        else:
            # Default transformation for masks
            mask_array = np.array(mask)
            mask_binary = (mask_array > 0).astype(np.int64)
            mask = torch.from_numpy(mask_binary).long()  # Explicit cast to long

        # For debugging: print data types and ranges (only for the first item)
        if idx == 0:
            print(f"Image shape: {image.shape}, dtype: {image.dtype}, range: [{image.min()}, {image.max()}]")
            print(f"Mask shape: {mask.shape}, dtype: {mask.dtype}, range: [{mask.min()}, {mask.max()}]")

        # Ensure mask is 2D (H,W) not 3D (1,H,W)
        if mask.dim() == 3 and mask.size(0) == 1:
            mask = mask.squeeze(0)

        return image, mask

# -------------------------
# Enhanced Selective Scan with Numerical Stability
# -------------------------
def selective_scan(u, delta, A, B, C, D):
    # Add numerical stability measures
    # Clamp A values to prevent extreme values
    A = torch.clamp(A, min=-5.0, max=5.0)

    dA = torch.einsum('bld,dn->bldn', delta, A)
    dB_u = torch.einsum('bld,bld,bln->bldn', delta, u, B)

    dA_cumsum = torch.cat([dA[:, 1:], torch.zeros_like(dA[:, :1])], dim=1)
    dA_cumsum = torch.flip(dA_cumsum, dims=[1])
    dA_cumsum = torch.cumsum(dA_cumsum, dim=1)
    # Prevent overflow in exponential
    dA_cumsum = torch.clamp(dA_cumsum, max=15.0)
    dA_cumsum = torch.exp(dA_cumsum)
    dA_cumsum = torch.flip(dA_cumsum, dims=[1])

    x = dB_u * dA_cumsum
    x = torch.cumsum(x, dim=1) / (dA_cumsum + 1e-6)

    y = torch.einsum('bldn,bln->bld', x, C)
    return y + u * D

# -------------------------
# Combined Loss Function
# -------------------------
class CombinedLoss(nn.Module):
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
        self.bce_loss = nn.CrossEntropyLoss()

    def forward(self, inputs, targets):
        # BCE Loss
        bce = self.bce_loss(inputs, targets)

        # Dice Loss
        inputs_soft = F.softmax(inputs, dim=1)
        targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[1]).permute(0, 3, 1, 2).float()

        # Calculate Dice loss manually
        intersection = (inputs_soft * targets_one_hot).sum(dim=(2, 3))
        cardinality = inputs_soft.sum(dim=(2, 3)) + targets_one_hot.sum(dim=(2, 3))

        dice = (2. * intersection / (cardinality + 1e-6)).mean()
        dice_loss = 1 - dice

        # Combined loss
        return self.bce_weight * bce + self.dice_weight * dice_loss

# -------------------------
# Improved MambaBlock with Gradient Checkpointing
# -------------------------
class MambaBlock(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.in_proj = nn.Linear(args.model_input_dims, args.model_internal_dim * 2, bias=False)
        self.conv1d = nn.Conv1d(args.model_internal_dim, args.model_internal_dim, kernel_size=args.conv_kernel_size,
                               padding=args.conv_kernel_size-1, groups=args.model_internal_dim)
        self.x_proj = nn.Linear(args.model_internal_dim, args.delta_t_rank + args.model_states * 2, bias=False)
        self.delta_proj = nn.Linear(args.delta_t_rank, args.model_internal_dim)

        # Initialize A values more carefully for better gradient flow
        A_vals = torch.arange(1, args.model_states + 1).float() / args.model_states * 3
        self.A_log = nn.Parameter(torch.log(repeat(A_vals, 'n -> d n', d=args.model_internal_dim)))
        self.D = nn.Parameter(torch.ones(args.model_internal_dim))
        self.out_proj = nn.Linear(args.model_internal_dim, args.model_input_dims, bias=args.dense_use_bias)

    def forward(self, x):
        # Use gradient checkpointing for better memory efficiency during training
        if self.training:
            return torch.utils.checkpoint.checkpoint(self._forward, x, use_reentrant=False)
        else:
            return self._forward(x)

    def _forward(self, x):
        b, l, d = x.shape
        x_and_res = self.in_proj(x)
        x1, res = x_and_res.chunk(2, dim=-1)

        x1 = rearrange(x1, 'b l d -> b d l')
        x1 = self.conv1d(x1)[..., :l]
        x1 = rearrange(x1, 'b d l -> b l d')
        x1 = F.silu(x1)

        # Apply bounded values for more stability
        A = -torch.exp(torch.clamp(self.A_log, min=-5, max=5))
        D = self.D
        x_dbl = self.x_proj(x1)
        delta, B, C = torch.split(x_dbl, [self.args.delta_t_rank, self.args.model_states, self.args.model_states], dim=-1)
        delta = F.softplus(self.delta_proj(delta))

        y = selective_scan(x1, delta, A, B, C, D)
        y = y * F.silu(res)
        return self.out_proj(y)

# -------------------------
# Enhanced Residual Block with Dropout and Layer Norm
# -------------------------
class ResidualBlock(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.norm1 = nn.LayerNorm(args.model_input_dims)
        self.mixer = MambaBlock(args)
        self.dropout = nn.Dropout(args.dropout_rate)
        self.norm2 = nn.LayerNorm(args.model_input_dims)

    def forward(self, x):
        # More transformer-like architecture with normalization
        residual = x
        x = self.norm1(x)
        x = self.mixer(x)
        x = self.dropout(x)
        x = residual + x
        return self.norm2(x)

# -------------------------
# Enhanced Double Convolution with BN and Residual Connection (No CBAM)
# -------------------------
class DoubleConvWithResidual(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.same_channels = in_channels == out_channels

        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

        # Optional projection for residual connection when channel sizes differ
        if not self.same_channels:
            self.project = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)

    def forward(self, x):
        identity = x if self.same_channels else self.project(x)
        x = self.double_conv(x)
        x = x + identity  # Residual connection
        return x

# -------------------------
# Improved Model Args for Better Performance
# -------------------------
class ModelArgs:
    def __init__(self):
        # Model dimensions
        self.model_input_dims = 96
        self.model_states = 96
        self.projection_expand_factor = 2
        self.conv_kernel_size = 4
        self.conv_use_bias = False
        self.dense_use_bias = False
        self.layer_id = -1
        self.seq_length = 256
        self.num_layers = 4
        self.dropout_rate = 0.2
        self.use_lm_head = False
        self.num_classes = 2  # Binary segmentation
        self.final_activation = 'none'
        self.model_internal_dim = self.projection_expand_factor * self.model_input_dims
        self.delta_t_rank = math.ceil(self.model_input_dims / 16)

# -------------------------
# Fixed Mamba-UNet Architecture with Deep Supervision (No CBAM)
# -------------------------
class MambaUNetSegmentationEnhanced(nn.Module):
    def __init__(self, args):
        super().__init__()

        # Encoder path with more channels
        self.encoder1 = DoubleConvWithResidual(3, 64)  # Input: 3 RGB channels
        self.pool1 = nn.MaxPool2d(2)

        self.encoder2 = DoubleConvWithResidual(64, 128)
        self.pool2 = nn.MaxPool2d(2)

        self.encoder3 = DoubleConvWithResidual(128, 256)
        self.pool3 = nn.MaxPool2d(2)

        # Mamba blocks in the bottleneck
        self.mamba_blocks = nn.Sequential(*[ResidualBlock(args) for _ in range(args.num_layers)])

        # Bridge between CNN and Mamba: Projection to adjust dimensions
        self.bridge_down = nn.Conv2d(256, args.model_input_dims, kernel_size=1)
        self.bridge_up = nn.Conv2d(args.model_input_dims, 256, kernel_size=1)

        # Decoder path with skip connections and deep supervision
        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder3 = DoubleConvWithResidual(256, 128)  # 256 = 128 + 128 (skip)
        self.deep_sup3 = nn.Conv2d(128, args.num_classes, kernel_size=1)

        self.upconv2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder2 = DoubleConvWithResidual(128, 64)  # 128 = 64 + 64 (skip)
        self.deep_sup2 = nn.Conv2d(64, args.num_classes, kernel_size=1)

        self.upconv1 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        # FIXED: Correct the channel dimension to match the concatenated input
        self.decoder1 = DoubleConvWithResidual(32 + 3, 32)  # 32+3 = 32 (from upconv1) + 3 (original input)

        # Final layer
        self.final_conv = nn.Conv2d(32, args.num_classes, kernel_size=1)

    def forward(self, x, return_deep=False):
        # Encoder path
        enc1 = self.encoder1(x)      # 64 channels
        enc1_pool = self.pool1(enc1)

        enc2 = self.encoder2(enc1_pool)  # 128 channels
        enc2_pool = self.pool2(enc2)

        enc3 = self.encoder3(enc2_pool)  # 256 channels
        enc3_pool = self.pool3(enc3)

        # Bridge to Mamba
        bridge_out = self.bridge_down(enc3_pool)

        # Reshape for Mamba blocks
        b, c, h, w = bridge_out.size()
        mamba_input = bridge_out.permute(0, 2, 3, 1).reshape(b, h * w, c)

        # Apply Mamba blocks
        mamba_output = self.mamba_blocks(mamba_input)

        # Reshape back to 2D
        mamba_output = mamba_output.reshape(b, h, w, c).permute(0, 3, 1, 2)

        # Bridge back to CNN
        mamba_output = self.bridge_up(mamba_output)

        # Decoder path with skip connections
        dec3 = self.upconv3(mamba_output)
        # Ensure dimensions match
        if dec3.shape[2:] != enc2.shape[2:]:
            dec3 = F.interpolate(dec3, size=enc2.shape[2:], mode='bilinear', align_corners=True)
        dec3 = torch.cat([dec3, enc2], dim=1)
        dec3 = self.decoder3(dec3)
        deep_out3 = self.deep_sup3(dec3)

        dec2 = self.upconv2(dec3)
        if dec2.shape[2:] != enc1.shape[2:]:
            dec2 = F.interpolate(dec2, size=enc1.shape[2:], mode='bilinear', align_corners=True)
        dec2 = torch.cat([dec2, enc1], dim=1)
        dec2 = self.decoder2(dec2)
        deep_out2 = self.deep_sup2(dec2)

        dec1 = self.upconv1(dec2)
        # Interpolate to match original input size and add skip connection to input
        if dec1.shape[2:] != x.shape[2:]:
            dec1 = F.interpolate(dec1, size=x.shape[2:], mode='bilinear', align_corners=True)
        dec1 = torch.cat([dec1, x], dim=1)  # Skip connection to original input
        dec1 = self.decoder1(dec1)

        # Final layer
        out = self.final_conv(dec1)

        if return_deep:
            # Return main output and deep supervision outputs
            deep_out2 = F.interpolate(deep_out2, size=x.shape[2:], mode='bilinear', align_corners=True)
            deep_out3 = F.interpolate(deep_out3, size=x.shape[2:], mode='bilinear', align_corners=True)
            return out, deep_out2, deep_out3

        return out

# -------------------------
# Deep Supervision Loss
# -------------------------
class DeepSupervisionLoss(nn.Module):
    def __init__(self, main_weight=0.6, deep2_weight=0.2, deep3_weight=0.2):
        super(DeepSupervisionLoss, self).__init__()
        self.main_weight = main_weight
        self.deep2_weight = deep2_weight
        self.deep3_weight = deep3_weight
        self.criterion = CombinedLoss(bce_weight=0.5, dice_weight=0.5)

    def forward(self, outputs, target):
        main_out, deep2, deep3 = outputs

        loss_main = self.criterion(main_out, target)
        loss_deep2 = self.criterion(deep2, target)
        loss_deep3 = self.criterion(deep3, target)

        total_loss = (
            self.main_weight * loss_main +
            self.deep2_weight * loss_deep2 +
            self.deep3_weight * loss_deep3
        )

        return total_loss

# -------------------------
# Evaluation Metrics
# -------------------------
def calculate_iou(pred_mask, gt_mask):
    """Calculate IoU for binary segmentation"""
    pred_mask = (pred_mask > 0).cpu().numpy().astype(bool)
    gt_mask = (gt_mask > 0).cpu().numpy().astype(bool)

    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()

    if union == 0:
        return 1.0  # If both masks are empty, IoU is 1

    return intersection / union

def calculate_dice(pred_mask, gt_mask):
    """Calculate Dice coefficient"""
    pred_mask = (pred_mask > 0).cpu().numpy().astype(bool)
    gt_mask = (gt_mask > 0).cpu().numpy().astype(bool)

    intersection = np.logical_and(pred_mask, gt_mask).sum()
    sum_areas = pred_mask.sum() + gt_mask.sum()

    if sum_areas == 0:
        return 1.0  # If both masks are empty, Dice is 1

    return 2.0 * intersection / sum_areas

# -------------------------
# Enhanced Training Function
# -------------------------
def train_one_epoch_enhanced(model, dataloader, optimizer, criterion, device, scheduler=None):
    model.train()
    running_loss = 0.0
    running_iou = 0.0
    running_dice = 0.0
    sample_count = 0

    pbar = tqdm(dataloader, desc='Training')

    for i, (images, masks) in enumerate(pbar):
        # Move data to device
        images = images.to(device)
        masks = masks.to(device)

        # Check data shape for the first batch
        if i == 0:
            print(f"Training batch - Images: {images.shape}, Masks: {masks.shape}")
            print(f"Masks unique values: {torch.unique(masks)}")

        # Forward pass with deep supervision
        outputs = model(images, return_deep=True)

        # Calculate loss with deep supervision
        loss = criterion(outputs, masks)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        # Optional gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # Step scheduler if provided
        if scheduler is not None:
            scheduler.step()

        # Get main output for metrics calculation
        main_output = outputs[0]

        # Calculate metrics
        batch_size = images.size(0)
        preds = torch.argmax(main_output, dim=1)

        # Update statistics
        running_loss += loss.item() * batch_size

        # Calculate metrics per image
        batch_iou = 0
        batch_dice = 0
        for j in range(batch_size):
            iou = calculate_iou(preds[j], masks[j])
            dice = calculate_dice(preds[j], masks[j])
            batch_iou += iou
            batch_dice += dice

        running_iou += batch_iou
        running_dice += batch_dice
        sample_count += batch_size

        # Update progress bar
        pbar.set_postfix({
            'loss': loss.item(),
            'iou': batch_iou / batch_size,
            'dice': batch_dice / batch_size
        })

        # Clear some GPU memory if needed
        del outputs, loss, preds
        if i % 10 == 0 and torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Calculate epoch statistics
    epoch_loss = running_loss / sample_count
    epoch_iou = running_iou / sample_count
    epoch_dice = running_dice / sample_count

    return epoch_loss, epoch_iou, epoch_dice

# -------------------------
# Validation Function
# -------------------------
def validate_enhanced(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_iou = 0.0
    running_dice = 0.0
    sample_count = 0

    with torch.no_grad():
        for images, masks in tqdm(dataloader, desc='Validation'):
            # Move data to device
            images = images.to(device)
            masks = masks.to(device)

            # Forward pass with deep supervision
            outputs = model(images, return_deep=True)

            # Calculate loss with deep supervision
            loss = criterion(outputs, masks)

            # Get main output for metrics calculation
            main_output = outputs[0]

            # Calculate metrics
            batch_size = images.size(0)
            preds = torch.argmax(main_output, dim=1)

            # Update statistics
            running_loss += loss.item() * batch_size

            # Calculate metrics per image
            for j in range(batch_size):
                iou = calculate_iou(preds[j], masks[j])
                dice = calculate_dice(preds[j], masks[j])
                running_iou += iou
                running_dice += dice

            sample_count += batch_size

    # Calculate statistics
    val_loss = running_loss / sample_count
    val_iou = running_iou / sample_count
    val_dice = running_dice / sample_count

    return val_loss, val_iou, val_dice

# -------------------------
# Visualization Function
# -------------------------
def visualize_results(model, dataloader, device, num_samples=3):
    model.eval()

    # Get a batch of data
    images, masks = next(iter(dataloader))
    images = images[:num_samples].to(device)
    masks = masks[:num_samples].to(device)

    # Generate predictions
    with torch.no_grad():
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)

    # Denormalize images for visualization
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).to(device)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).to(device)
    images = images * std + mean

    # Create figure with subplots
    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4 * num_samples))

    for i in range(num_samples):
        # Display original image
        axes[i, 0].imshow(images[i].permute(1, 2, 0).cpu().numpy())
        axes[i, 0].set_title("Original Image")
        axes[i, 0].axis("off")

        # Display ground truth mask
        axes[i, 1].imshow(masks[i].cpu().numpy(), cmap="gray")
        axes[i, 1].set_title("Ground Truth")
        axes[i, 1].axis("off")

        # Display predicted mask
        axes[i, 2].imshow(predictions[i].cpu().numpy(), cmap="gray")
        axes[i, 2].set_title("Prediction")
        axes[i, 2].axis("off")

    plt.tight_layout()
    plt.savefig("mamba_segmentation_nocbam_results.png")
    plt.show()

# -------------------------
# Main Function
# -------------------------
def main():
    print("Starting Mamba-UNet for Kvasir-SEG polyp segmentation (No CBAM, 100 Epochs)...")

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Set seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)

    # Download and set up the dataset
    kvasir_path = './datasets/kvasir-seg'
    if not os.path.exists(os.path.join(kvasir_path, 'images')):
        kvasir_path = download_and_setup_dataset(force_download=False)

    if not kvasir_path:
        print("Dataset setup failed. Exiting...")
        return

    # Define transformations
    transform = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    target_transform = T.Compose([
        T.Resize((256, 256), interpolation=T.InterpolationMode.NEAREST),
        T.ToTensor(),
        lambda x: (x > 0.5).long()
    ])

    # Create datasets and data loaders
    try:
        train_dataset = KvasirSEGDataset(
            kvasir_path,
            split='train',
            transform=transform,
            target_transform=target_transform,
            augment=True
        )

        val_dataset = KvasirSEGDataset(
            kvasir_path,
            split='val',
            transform=transform,
            target_transform=target_transform,
            augment=False
        )

        # Use smaller batch size for more stable training
        train_loader = DataLoader(
            train_dataset,
            batch_size=4,
            shuffle=True,
            num_workers=2,
            pin_memory=True if torch.cuda.is_available() else False
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=4,
            shuffle=False,
            num_workers=2,
            pin_memory=True if torch.cuda.is_available() else False
        )

        print("Data loaders created successfully.")
    except Exception as e:
        print(f"Error creating datasets: {e}")
        import traceback
        traceback.print_exc()
        return

    # Initialize enhanced model args and create model
    args = ModelArgs()
    model = MambaUNetSegmentationEnhanced(args).to(device)
    print("Enhanced Mamba-UNet model created without CBAM attention.")

    # Define enhanced loss function and optimizer
    criterion = DeepSupervisionLoss(main_weight=0.6, deep2_weight=0.2, deep3_weight=0.2)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

    # Add learning rate scheduler with warm restarts - adjusted for 100 epochs
    scheduler = CosineAnnealingWarmRestarts(
        optimizer,
        T_0=10,  # Restart every 10 epochs
        T_mult=2,  # Double period after each restart
        eta_min=1e-6,
    )

    # Training loop with increased epochs
    num_epochs = 100  # Increased to 100 as requested
    best_iou = 0.0
    patience_counter = 0
    max_patience = 20  # Early stopping after 20 epochs without improvement

    history = {
        'train_loss': [], 'train_iou': [], 'train_dice': [],
        'val_loss': [], 'val_iou': [], 'val_dice': []
    }

    print(f"Starting training for {num_epochs} epochs...")

    try:
        for epoch in range(num_epochs):
            print(f"Epoch {epoch+1}/{num_epochs}")

            # Train with enhanced functions
            train_loss, train_iou, train_dice = train_one_epoch_enhanced(
                model, train_loader, optimizer, criterion, device, scheduler
            )

            # Validate
            val_loss, val_iou, val_dice = validate_enhanced(
                model, val_loader, criterion, device
            )

            # Save history
            history['train_loss'].append(train_loss)
            history['train_iou'].append(train_iou)
            history['train_dice'].append(train_dice)
            history['val_loss'].append(val_loss)
            history['val_iou'].append(val_iou)
            history['val_dice'].append(val_dice)

            # Print epoch results
            print(f"Train - Loss: {train_loss:.4f}, IoU: {train_iou:.4f}, Dice: {train_dice:.4f}")
            print(f"Val   - Loss: {val_loss:.4f}, IoU: {val_iou:.4f}, Dice: {val_dice:.4f}")

            # Save best model
            if val_iou > best_iou:
                best_iou = val_iou
                torch.save(model.state_dict(), "best_mamba_unet_nocbam_model.pth")
                print(f"Model saved with IoU: {best_iou:.4f}")
                patience_counter = 0  # Reset patience counter
            else:
                patience_counter += 1

            # Save checkpoint every 10 epochs for safety
            if (epoch+1) % 10 == 0:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'best_iou': best_iou,
                    'history': history,
                }, f"checkpoint_nocbam_epoch_{epoch+1}.pth")

                # Plot and save training progress
                plot_training_progress(history, epoch+1)

            # Early stopping
            if patience_counter >= max_patience:
                print(f"Early stopping after {max_patience} epochs without improvement")
                break

            # Clear GPU cache between epochs
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    except Exception as e:
        print(f"Error during training: {e}")
        import traceback
        traceback.print_exc()

        # Save checkpoint on error
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
            'best_iou': best_iou,
            'history': history,
        }, "error_checkpoint_nocbam.pth")

    # Load best model and visualize results
    try:
        print("Loading best model for visualization...")
        model.load_state_dict(torch.load("best_mamba_unet_nocbam_model.pth"))
        visualize_results(model, val_loader, device)

        # Plot final training history
        plot_training_progress(history, num_epochs, final=True)

        print(f"Final best IoU: {best_iou:.4f}")

    except Exception as e:
        print(f"Error during visualization: {e}")
        import traceback
        traceback.print_exc()

def plot_training_progress(history, epoch, final=False):
    """Plot and save training progress"""
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Train Loss', marker='o')
    plt.plot(history['val_loss'], label='Val Loss', marker='s')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss Value')
    plt.grid(True)
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(history['train_iou'], label='Train IoU', marker='o')
    plt.plot(history['val_iou'], label='Val IoU', marker='s')
    plt.title('IoU')
    plt.xlabel('Epoch')
    plt.ylabel('IoU Score')
    plt.grid(True)
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(history['train_dice'], label='Train Dice', marker='o')
    plt.plot(history['val_dice'], label='Val Dice', marker='s')
    plt.title('Dice Coefficient')
    plt.xlabel('Epoch')
    plt.ylabel('Dice Score')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()

    if final:
        plt.savefig("mamba_unet_nocbam_final_training_history.png")
    else:
        plt.savefig(f"mamba_unet_nocbam_training_history_epoch_{epoch}.png")

    plt.close()

if __name__ == "__main__":
    main()

Starting Mamba-UNet for Kvasir-SEG polyp segmentation (No CBAM, 100 Epochs)...
Using device: cuda


100%|██████████| 46.2M/46.2M [00:03<00:00, 14.1MiB/s]


Download completed, file saved to ./datasets/kvasir-seg.zip
Extracting dataset...
Extraction completed.
Extracted file structure:
  ./datasets/
  ./datasets/kvasir-seg.zip
  ./datasets/Kvasir-SEG
  ./datasets/Kvasir-SEG/kavsir_bboxes.json
  ./datasets/Kvasir-SEG/images
  ./datasets/Kvasir-SEG/images/ck2395w2mb4vu07480otsu6tw.jpg
  ./datasets/Kvasir-SEG/images/cju2qh5le1ock0878oahaql7d.jpg
  ./datasets/Kvasir-SEG/images/cju35mdz73x890835eynq1h9v.jpg
  ./datasets/Kvasir-SEG/images/cju2qdj95ru8g09886gfi9rsz.jpg
  ./datasets/Kvasir-SEG/images/cju2z9vlp9j0w0801oag91sy9.jpg
  ... and 1996 more files
Found image directories: ['./datasets/Kvasir-SEG/images']
Found mask directories: ['./datasets/Kvasir-SEG/masks']
Moving images from ./datasets/Kvasir-SEG/images to ./datasets/kvasir-seg/images
Moving masks from ./datasets/Kvasir-SEG/masks to ./datasets/kvasir-seg/masks
Removed zip file.
Dataset setup completed successfully.
Found 1000 images and 1000 masks.
Sample image names: ['cju0qkwl35piu099

Training:   0%|          | 0/200 [00:00<?, ?it/s]

Training batch - Images: torch.Size([4, 3, 256, 256]), Masks: torch.Size([4, 256, 256])
Masks unique values: tensor([0, 1], device='cuda:0')


...

Validation: 100%|██████████| 25/25 [00:05<00:00,  4.80it/s]


Train - Loss: 0.1563, IoU: 0.6900, Dice: 0.7903
Val   - Loss: 0.2039, IoU: 0.6065, Dice: 0.7094
Epoch 53/100


Training:   0%|          | 0/200 [00:00<?, ?it/s]

Training batch - Images: torch.Size([4, 3, 256, 256]), Masks: torch.Size([4, 256, 256])
Masks unique values: tensor([0, 1], device='cuda:0')


Training:  12%|█▎        | 25/200 [00:20<02:21,  1.24it/s, loss=0.108, iou=0.742, dice=0.837]